<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/Evaluation1.0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 42.7 MB/s 
     |████████████████████████████████| 61 kB 391 kB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 50 kB 2.8 MB/s 


In [1]:
# !git clone https://github.com/meti-94/OpenQA.git
!python ./OpenQA/src/EntityLinking.py 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Reverb Questions Count: 5003	Freebase Questions Count: 21619
100% 407267/407267 [00:50<00:00, 8090.45it/s]

Total MIDs before augmentation: 1951909	Unmatched (Added) MIDs: 669962	 Matched MIDs: 144572
Total Entity Strings before augmentation: 17555942	Added Entity Strings: 138167
Total type of text: 4936783
Max Length of entry is 249717, text is [('fb:m.04b3t_p', 'ralston , mississippi', 'fb:common.topic.alias'), ('fb:m.02q07z', 'tokyo metropolitan college ( public ) , main campus', 'fb:common.topic.alias'), ('fb:m.0sdztd', "may it please you , m'lud", 'fb:type.object.name'), ('fb:m.073fq6', 'president kennedy school and community college , main campus', 'fb:common.topic.alias'), ('fb:m.0x544', 'branford center , connecticut', 'fb:type.object.name'), ('fb:m.05byvg', 'general motors building , detroit', 'fb:common.topic.alias'), ('fb:m.027_sn', "william december `` bill

In [49]:
import pandas as pd
import pickle
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
entity_df = pd.read_excel('/content/Linking.xlsx')
relation_df = pd.read_excel('/content/drive/MyDrive/data_freebase/dbg.xlsx')
predictions = pd.merge(entity_df, relation_df, on='Question').drop_duplicates()
reverb_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test.xlsx')[['Question', 'triple', 'Reverb_no']]
reverb_df['Relation'] = reverb_df['triple'].apply(lambda item:eval(item)[1])
reverb_df = reverb_df[['Question', 'Relation', 'Reverb_no']]
freebase_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test_useful_records.xlsx')[['Question', 'relation_type']]
freebase_df['Relation'] = freebase_df['relation_type']
freebase_df['Reverb_no'] = freebase_df['relation_type'].apply(lambda item:'')
freebase_df = freebase_df[['Question', 'Relation', 'Reverb_no']]
actuals = pd.concat([reverb_df, freebase_df])

In [4]:
data = pd.merge(actuals, predictions, on='Question', how='inner')
data.head()

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb
0,how do Protocols vary,vary by,267285,protocols,5f5d3e85-c87f-4fa2-b636-5ce2d89acab5,10,"[(('b31be1a0-0c38-4227-bdd0-5a78f2196253', 'pr...","[('fb:medicine.disease.medical_specialties', 0...","{""do n't vary based on"": 0.84525030966336, 'do..."
1,of what is A Shooting Star another brand,is another brand of,8600,a shooting star,254a68ac-99ca-4a53-8d2a-b841ac0fbbe2,3,"[(('305bf346-b11f-4cd0-8f21-f9e58ea327c1', 'a ...","[('fb:automotive.model.successor', 0.043253843...","{'is another brand of': 1.0000000000000002, 'i..."
2,where was the hometown of Angie Debo,was the hometown of,212931,angie debo,baf4dcad-f15d-40ea-9517-5254ab996785,1,"[(('baf4dcad-f15d-40ea-9517-5254ab996785', 'an...","[('fb:music.artist.origin', 0.861007), ('fb:pe...","{'was the hometown of': 1.0000000000000002, 'i..."
3,where Curium does not occur,does not occur in,89850,curium,fb:m.025tkv0,1,"[(('fb:m.025tkv0', 'curium', 'fb:type.object.n...",[('fb:theater.theater_production.performed_at'...,"{'does not occur in': 0.986554414128868, 'does..."
4,who died in Moraga,died in,385331,moraga,a1fd5fa8-d7db-4a01-877d-7f6a336b768e,3,"[(('fb:m.0qysj', 'moraga', '0.94124'), 1.0), (...","[('fb:location.location.people_born_here', 0.8...","{'died in': 1.0000000000000002, 'Died in': 1.0..."


In [88]:
data.tail()

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb,answers
26643,"name a composer in the album , `` the great co...",fb:music.album.artist,,", `` the great composers iv '' .",fb:m.0dshw5z,-1,"[(('fb:m.016q49', ""`` the great imposter ''"", ...","[('fb:music.album.artist', 0.98206264), ('fb:m...","{'is the composer of': 0.6999592681181698, 'wa...","[(fb:m.0fy0t0x, fb:music.album.artist, 0.78, 0..."
26644,what artist recorded colors,fb:music.recording.artist,,colors,fb:m.0_d3jh,-1,"[(('fb:m.05c_00d', 'colors', 'fb:type.object.n...","[('fb:music.recording.artist', 0.9823801), ('f...","{'is an artist in': 0.6956105793832068, 'was a...","[(fb:m.0qw7xk, fb:music.recording.artist, 1.0,..."
26645,what celestial object is 2974 holden,fb:astronomy.celestial_object.category,,2974 holden,fb:m.03y3tdm,1,"[(('fb:m.03y3tdm', '2974 holden', 'fb:type.obj...","[('fb:astronomy.celestial_object.category', 0....","{'object to': 0.9439233954093864, 'is the obje...","[(fb:m.03y3tdm, fb:astronomy.celestial_object...."
26646,what is the film genre for snow falling on ced...,fb:film.film.genre,,snow falling on cedars,fb:m.060__7,3,"[(('fb:m.060_3t', 'snow falling on cedars', 'f...","[('fb:film.film.genre', 0.97367465), ('fb:medi...","{'is the genre of': 0.7384591683301363, 'is a ...","[(fb:m.060__7, fb:film.film.genre, 1.0, 0.9736..."
26647,what nationality is lucille clifton ?,fb:people.person.nationality,,lucille clifton,fb:m.0695_d,1,"[(('fb:m.0695_d', 'lucille clifton', 'fb:type....","[('fb:people.person.nationality', 0.99201274),...","{""is n't"": 1.0, ""Is n't"": 1.0, ""is 'n"": 1.0, '...","[(fb:m.0695_d, fb:people.person.nationality, 1..."


In [5]:
reverb2freebace = pd.read_csv('/content/drive/MyDrive/reverb2freebase.csv')
reverb2freebace['freebase_ID_argument1'] = reverb2freebace['freebase_ID_argument1'].apply(lambda string:'fb:m.'+str(string))
reverb2freebace['conf'] = reverb2freebace['conf'].astype(float)
# reverb2freebace.info()

In [6]:
# mapping between MIDs and names in the form of dict['MID']=['str1', 'str2', ...,  'strN']
with open('/content/drive/MyDrive/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)
# mapping between MIDs and Relations in the form of dict['MID']=[{'fb:common.topic.notable_types', 'fb:people.person.gender', 'fb:people.person.profession'}]
with open('/content/drive/MyDrive/indexes/reachability_2M.pkl', 'rb') as f:
    reachability_2M = pickle.load(f)
print(len(reachability_2M.keys()), sum([len(item) for item in reachability_2M.values()]))

1959805 7188598


In [7]:
def combine(reachability_2M, reverb2freebace):
  for index, row in tqdm(reverb2freebace.iterrows(), total=reverb2freebace.shape[0]):
    if row['freebase_ID_argument1'] in mid2name:
      mid1 = mid = row['freebase_ID_argument1'] 
    else:
      mid1 = mid = row['argument1_uuid']
    mid2 = mid = row['argument2_uuid']
    relation = row['rel']
    reverb_no = row['reverb_no']
    try:
      temp = reachability_2M[mid1]
    except:
      reachability_2M[mid1] = set()
      temp = reachability_2M[mid1]
    temp.add((relation, reverb_no))
    try:
      temp = reachability_2M[mid2]
    except:
      reachability_2M[mid2] = set()
      temp = reachability_2M[mid2]
    temp.add((relation, reverb_no))
  return reachability_2M


In [8]:
reachability_2M = combine(reachability_2M, reverb2freebace)
print(len(reachability_2M.keys()), sum([len(item) for item in reachability_2M.values()]))

100%|██████████| 407267/407267 [00:49<00:00, 8283.03it/s]


2558314 7994311


In [85]:
def create_candidates(row):
  ner = row['Candidates']
  freebase = row['Freebase']
  reverb = row['Reverb']
  candidates = []
  rvb = [(key, value) for key, value in eval(reverb).items()]
  rels = eval(freebase)+rvb
  try:
    for idx1, ((mid, string, conf), sim1) in enumerate(eval(ner)):
        relations = list(reachability_2M[mid])
        for relation in relations:
          if isinstance(relation, tuple):
            for idx2, (rel, sim2) in enumerate(rels):
              if relation[0]==rel:
                candidates.append((mid, rel, sim1, sim2, conf, relation[1]))
          else:
            for idx2, (rel, sim2) in enumerate(rels):
              if relation==rel:
                candidates.append((mid, rel, sim1, sim2, conf))
    return sorted(candidates, key=lambda item:item[2]+item[3], reverse=True)
  except:
    return None

In [86]:
# test = data.loc[0, 'Candidates']
data['answers'] = data.progress_apply(create_candidates, axis=1)
# create_candidates(data.loc[4900])
# create_candidates(data.loc[0, 'Candidates'], data.loc[0, 'Freebase'], data.loc[0, 'Reverb'])

  0%|          | 0/26648 [00:00<?, ?it/s]

In [92]:
data = data.dropna(subset=['answers'])
data.answers.isnull().sum()

0

In [113]:
data

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb,answers
0,how do Protocols vary,vary by,267285,protocols,5f5d3e85-c87f-4fa2-b636-5ce2d89acab5,10,"[(('b31be1a0-0c38-4227-bdd0-5a78f2196253', 'pr...","[('fb:medicine.disease.medical_specialties', 0...","{""do n't vary based on"": 0.84525030966336, 'do...","[(5f5d3e85-c87f-4fa2-b636-5ce2d89acab5, vary b..."
1,of what is A Shooting Star another brand,is another brand of,8600,a shooting star,254a68ac-99ca-4a53-8d2a-b841ac0fbbe2,3,"[(('305bf346-b11f-4cd0-8f21-f9e58ea327c1', 'a ...","[('fb:automotive.model.successor', 0.043253843...","{'is another brand of': 1.0000000000000002, 'i...","[(254a68ac-99ca-4a53-8d2a-b841ac0fbbe2, is ano..."
2,where was the hometown of Angie Debo,was the hometown of,212931,angie debo,baf4dcad-f15d-40ea-9517-5254ab996785,1,"[(('baf4dcad-f15d-40ea-9517-5254ab996785', 'an...","[('fb:music.artist.origin', 0.861007), ('fb:pe...","{'was the hometown of': 1.0000000000000002, 'i...","[(baf4dcad-f15d-40ea-9517-5254ab996785, was th..."
3,where Curium does not occur,does not occur in,89850,curium,fb:m.025tkv0,1,"[(('fb:m.025tkv0', 'curium', 'fb:type.object.n...",[('fb:theater.theater_production.performed_at'...,"{'does not occur in': 0.986554414128868, 'does...","[(fb:m.025tkv0, does not occur in, 1.0, 0.9865..."
4,who died in Moraga,died in,385331,moraga,a1fd5fa8-d7db-4a01-877d-7f6a336b768e,3,"[(('fb:m.0qysj', 'moraga', '0.94124'), 1.0), (...","[('fb:location.location.people_born_here', 0.8...","{'died in': 1.0000000000000002, 'Died in': 1.0...","[(a1fd5fa8-d7db-4a01-877d-7f6a336b768e, died i..."
...,...,...,...,...,...,...,...,...,...,...
26643,"name a composer in the album , `` the great co...",fb:music.album.artist,,", `` the great composers iv '' .",fb:m.0dshw5z,-1,"[(('fb:m.016q49', ""`` the great imposter ''"", ...","[('fb:music.album.artist', 0.98206264), ('fb:m...","{'is the composer of': 0.6999592681181698, 'wa...","[(fb:m.0fy0t0x, fb:music.album.artist, 0.78, 0..."
26644,what artist recorded colors,fb:music.recording.artist,,colors,fb:m.0_d3jh,-1,"[(('fb:m.05c_00d', 'colors', 'fb:type.object.n...","[('fb:music.recording.artist', 0.9823801), ('f...","{'is an artist in': 0.6956105793832068, 'was a...","[(fb:m.0qw7xk, fb:music.recording.artist, 1.0,..."
26645,what celestial object is 2974 holden,fb:astronomy.celestial_object.category,,2974 holden,fb:m.03y3tdm,1,"[(('fb:m.03y3tdm', '2974 holden', 'fb:type.obj...","[('fb:astronomy.celestial_object.category', 0....","{'object to': 0.9439233954093864, 'is the obje...","[(fb:m.03y3tdm, fb:astronomy.celestial_object...."
26646,what is the film genre for snow falling on ced...,fb:film.film.genre,,snow falling on cedars,fb:m.060__7,3,"[(('fb:m.060_3t', 'snow falling on cedars', 'f...","[('fb:film.film.genre', 0.97367465), ('fb:medi...","{'is the genre of': 0.7384591683301363, 'is a ...","[(fb:m.060__7, fb:film.film.genre, 1.0, 0.9736..."


In [119]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0
for idx1, row in data.iterrows():
  for idx2, candidate in enumerate(row['answers']):
    if len(candidate)==6:
      if candidate[-1]==row['Reverb_no']:
        if idx2 in range(1):
            # print(candidate, row['Reverb_no'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break 
    else:
      if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
        if idx2 in range(1):
            # print(candidate, row['Answer'], row['Relation'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break 
   

# print(data_type)
total = len(data)
print("Top1 Answers: {}".format(top1 / total))
print("Top3 Answers: {}".format(top3 / total))
print("Top5 Answers: {}".format(top5 / total))
print("Top10 Answers: {}".format(top10 / total))
print("Top20 Answers: {}".format(top20 / total))
print("Top50 Answers: {}".format(top50 / total))
print("Top100 Answers: {}".format(top100 / total))

Top1 Answers: 0.7106882834196502
Top3 Answers: 0.7901748855362907
Top5 Answers: 0.8092771898221122
Top10 Answers: 0.8256773999849883
Top20 Answers: 0.8353223748405014
Top50 Answers: 0.8427906627636418
Top100 Answers: 0.845117466036178


In [109]:
data.head(19)

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb,answers
0,how do Protocols vary,vary by,267285,protocols,5f5d3e85-c87f-4fa2-b636-5ce2d89acab5,10,"[(('b31be1a0-0c38-4227-bdd0-5a78f2196253', 'pr...","[('fb:medicine.disease.medical_specialties', 0...","{""do n't vary based on"": 0.84525030966336, 'do...","[(5f5d3e85-c87f-4fa2-b636-5ce2d89acab5, vary b..."
1,of what is A Shooting Star another brand,is another brand of,8600,a shooting star,254a68ac-99ca-4a53-8d2a-b841ac0fbbe2,3,"[(('305bf346-b11f-4cd0-8f21-f9e58ea327c1', 'a ...","[('fb:automotive.model.successor', 0.043253843...","{'is another brand of': 1.0000000000000002, 'i...","[(254a68ac-99ca-4a53-8d2a-b841ac0fbbe2, is ano..."
2,where was the hometown of Angie Debo,was the hometown of,212931,angie debo,baf4dcad-f15d-40ea-9517-5254ab996785,1,"[(('baf4dcad-f15d-40ea-9517-5254ab996785', 'an...","[('fb:music.artist.origin', 0.861007), ('fb:pe...","{'was the hometown of': 1.0000000000000002, 'i...","[(baf4dcad-f15d-40ea-9517-5254ab996785, was th..."
3,where Curium does not occur,does not occur in,89850,curium,fb:m.025tkv0,1,"[(('fb:m.025tkv0', 'curium', 'fb:type.object.n...",[('fb:theater.theater_production.performed_at'...,"{'does not occur in': 0.986554414128868, 'does...","[(fb:m.025tkv0, does not occur in, 1.0, 0.9865..."
4,who died in Moraga,died in,385331,moraga,a1fd5fa8-d7db-4a01-877d-7f6a336b768e,3,"[(('fb:m.0qysj', 'moraga', '0.94124'), 1.0), (...","[('fb:location.location.people_born_here', 0.8...","{'died in': 1.0000000000000002, 'Died in': 1.0...","[(a1fd5fa8-d7db-4a01-877d-7f6a336b768e, died i..."
5,what also offers Perst,also offers,218200,perst,5bad996e-3852-4b79-bdf4-227f0e2dbb3e,1,"[(('5bad996e-3852-4b79-bdf4-227f0e2dbb3e', 'pe...","[('fb:computer.software.license', 0.12468366),...","{'also offers': 1.0, 'offers': 0.8633380973583...","[(5bad996e-3852-4b79-bdf4-227f0e2dbb3e, also o..."
6,who was a co-founder of the Natural Resources ...,was a co-founder of,308885,natural resources defense council,fb:m.033bjq,-1,"[(('fb:m.0fzvn', 'natural resources defense co...","[('fb:organization.organization.founders', 0.9...","{'was a co-founder of': 1.0, 'was co-founder o...","[(fb:m.0fzvn, fb:organization.organization.fou..."
7,who became a star in the American Hockey League,became a star in,217531,the american hockey league,f7dbe751-8f42-4a36-9c1d-5de25492c850,-1,"[(('fb:m.07k2bw', 'north american hockey leagu...","[('fb:ice_hockey.hockey_position.players', 0.6...","{'became a star in': 1.0, 'became a star with'...",[]
8,When were Personal ranks reintroduced,were reintroduced in,256000,personal ranks,db04c1c8-efb0-447c-9181-4dd25897dcf3,1,"[(('db04c1c8-efb0-447c-9181-4dd25897dcf3', 'pe...","[('fb:music.release.region', 0.014240686), ('f...","{'were reintroduced in': 0.9864233809915312, '...","[(db04c1c8-efb0-447c-9181-4dd25897dcf3, were r..."
9,By whom was Zen Habits created,was created by,392400,zen habits,fb:m.051x_my,1,"[(('fb:m.051x_my', 'zen habits', 'fb:type.obje...","[('fb:visual_art.artwork.artist', 0.21437937),...","{'was created by': 1.0, 'is created by': 0.926...","[(fb:m.051x_my, was created by, 1.0, 1.0, fb:t..."


In [102]:
len(data)

26646

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
